# Выбор локации для скважины

Допустим, мы работаете в добывающей компании «ГлавРосГосНефть». Нам нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
#Импортируем библиотеку Pandas.
import pandas as pd
from pandas import DataFrame
#Импортируем библиотеку для постраения графиков matplotlib.pyplot.
import matplotlib.pyplot as plt
#Импортируем библиотеку для постраения графиков seaborn.
import seaborn as sns
#Импортируем библиотеку для высокоуровневых математических функций numpy.
import numpy as np
import math
# Импортируем библиотеки машинного обучения 
from scipy import stats as st 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

In [2]:
#Прочитаем исходные DataFrame.
try:
    df_0 = pd.read_csv('/datasets/geo_data_0.csv')
    df_1 = pd.read_csv('/datasets/geo_data_1.csv')
    df_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    df_0 = pd.read_csv('geo_data_0.csv')
    df_1 = pd.read_csv('geo_data_1.csv')
    df_2 = pd.read_csv('geo_data_2.csv')

In [3]:
df_0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [4]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df_1

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [6]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
df_2

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [8]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


### Опишим данные<br>
**Признаки**<br>
***id*** — уникальный идентификатор скважины<br>
***f0, f1, f2*** — три признака точек (неважно, что они означают, но сами признаки значимы)<br>
***product*** — объём запасов в скважине (тыс. баррелей)<br>

In [9]:
# Извлечем признаки и целевое знаечение
features_0 = df_0.drop(['product', 'id'], axis=1)
target_0 = df_0['product']

In [10]:
# Разделим обучающую, валидационную 75:25
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)

In [11]:
# Проверим размеры выборок
print('Размер выборки train:', features_train_0.shape)
print('Размер выборки valid:', features_valid_0.shape)

Размер выборки train: (75000, 3)
Размер выборки valid: (25000, 3)


In [12]:
# Извлечем признаки и целевое знаечение
features_1 = df_1.drop(['product', 'id'], axis=1)
target_1 = df_1['product']

In [13]:
# Разделим обучающую, валидационную 75:25
features_train_1, features_valid_1, target_train_1, target_valid_1= train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

In [14]:
# Проверим размеры выборок
print('Размер выборки train:', features_train_1.shape)
print('Размер выборки valid:', features_valid_1.shape)

Размер выборки train: (75000, 3)
Размер выборки valid: (25000, 3)


In [15]:
# Извлечем признаки и целевое знаечение
features_2 = df_2.drop(['product', 'id'], axis=1)
target_2 = df_2['product']

In [16]:
# Разделим обучающую, валидационную 75:25
features_train_2, features_valid_2, target_train_2, target_valid_2= train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

In [17]:
# Проверим размеры выборок
print('Размер выборки train:', features_train_2.shape)
print('Размер выборки valid:', features_valid_2.shape)

Размер выборки train: (75000, 3)
Размер выборки valid: (25000, 3)


## Обучение и проверка модели

### Регион geo_data_0

In [18]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predicted_valid_0 = model_0.predict(features_valid_0) 

In [19]:
# Средний запас предсказанного сырья
predicted_valid_0.mean()

92.59256778438035

In [20]:
# RMSE модели
mse = mean_squared_error(target_valid_0, predicted_valid_0)
print("MSE_0 =", mse)
print("RMSE_0 =", mse ** 0.5)

MSE_0 = 1412.2129364399243
RMSE_0 = 37.5794217150813


In [21]:
print("R2 =", r2_score(target_valid_0, predicted_valid_0))

R2 = 0.27994321524487786


### Регион geo_data_1

In [22]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predicted_valid_1 = model_1.predict(features_valid_1) 

In [23]:
# Средний запас предсказанного сырья
predicted_valid_1.mean()

68.728546895446

In [24]:
# RMSE модели
mse = mean_squared_error(target_valid_1, predicted_valid_1)
print("MSE_0 =", mse)
print("RMSE_0 =", mse ** 0.5)

MSE_0 = 0.797626336039116
RMSE_0 = 0.8930992867756171


In [25]:
print("R2 =", r2_score(target_valid_1, predicted_valid_1))

R2 = 0.9996233978805127


### Регион geo_data_2

In [26]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predicted_valid_2 = model_2.predict(features_valid_2) 

In [27]:
# Средний запас предсказанного сырья
predicted_valid_2.mean()

94.96504596800489

In [28]:
# RMSE модели
mse = mean_squared_error(target_valid_2, predicted_valid_2)
print("MSE_0 =", mse)
print("RMSE_0 =", mse ** 0.5)

MSE_0 = 1602.3775813236196
RMSE_0 = 40.02970873393434


In [29]:
print("R2 =", r2_score(target_valid_2, predicted_valid_2))

R2 = 0.20524758386040443


### Вывод

- Мы видим, что в регионе geo_data_2 самый большой средний запас (94.96), но и самый высокий показатель RMSE модели составляет 40.02 - это очень высокая ошибка.<br>
- В регионе geo_data_1 средний запас самый низкий (68.72), но и ошибка составляет всего (0.89) - что показывает высокую точность.<br>

## Подготовка к расчёту прибыли

In [30]:
# Бюджен на разработку скважин 10 000 000 000 или 10 000 000 тыс.руб.
BUDGET = 10000000000
# Цена 1 единицы (1000 бареллей) составляет 450 000 или 450 тыс.руб.
PRICE = 450000
# Количество планируемых скважин
COUNT = 200

In [31]:
# Минимальный объем сырья на 200 лучших скважинах должен быть
min_value = BUDGET / PRICE
min_value

22222.222222222223

In [32]:
# В среднем на 1 скважину для безубыточной работы
print(min_value / COUNT)

111.11111111111111


In [33]:
# В среднем на 1 скважину для безубыточной работы
income = (min_value / COUNT) * PRICE
print(income)

50000000.0


In [34]:
# Средний запас в каждом регионе
print(df_0['product'].mean())
print(df_1['product'].mean())
print(df_2['product'].mean())

92.50000000000001
68.82500000000002
95.00000000000004


Мы видим, что средний запас в каждом регионе ниже, чем в среднем нам необходимо на скважину.

## Расчёт прибыли и рисков 

In [35]:
# Сделаем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели
def profit(target, well, count):
    well_sorted = well.sort_values(ascending=False)
    selected = target[well_sorted.index][:count]
    return round(selected.sum()*PRICE - income*COUNT)

In [36]:
# Сохраним результаты предсказаний в объекты dataframe 
well_0=pd.DataFrame(model_0.predict(features_valid_0))
well_0=well_0[0]
well_1=pd.DataFrame(model_1.predict(features_valid_1))
well_1=well_1[0]
well_2=pd.DataFrame(model_1.predict(features_valid_2))
well_2=well_2[0]
target_0 = target_valid_0.reset_index(drop=True)
target_1 = target_valid_1.reset_index(drop=True)
target_2 = target_valid_2.reset_index(drop=True)

In [37]:
# Для сравнения рассчитаем прибыль по 200 ТОП, с сортировкой по реальному значению продукта:
print('Прибыль на', COUNT, 'ТОП-скважин в регионе 0: ',profit(target_0,target_0,COUNT))
print('Прибыль на', COUNT, 'ТОП-скважин в регионе 1: ',profit(target_1,target_1,COUNT))
print('Прибыль на', COUNT, 'ТОП-скважин в регионе 2: ',profit(target_2,target_2,COUNT))

Прибыль на 200 ТОП-скважин в регионе 0:  6510947580
Прибыль на 200 ТОП-скважин в регионе 1:  2415086697
Прибыль на 200 ТОП-скважин в регионе 2:  6942663159


In [38]:
# Выберем 200 ТОП скважин на основе прогнозного продукта и рассчитаем прибыль по реальному продукту этих скважин.
print('Прибыль на', COUNT, 'ТОП-скважин в регионе 0: ',profit(target_0, well_0,COUNT))
print('Прибыль на', COUNT, 'ТОП-скважин в регионе 1: ',profit(target_1,well_1,COUNT))
print('Прибыль на', COUNT, 'ТОП-скважин в регионе 2: ',profit(target_2,well_2,COUNT))

Прибыль на 200 ТОП-скважин в регионе 0:  3320826043
Прибыль на 200 ТОП-скважин в регионе 1:  2415086697
Прибыль на 200 ТОП-скважин в регионе 2:  2737074693


In [39]:
# Сделаем список DF из регионов
predict=[well_0,well_1,well_2]
target=[target_0,target_1,target_2]

# Переберем циклом регины и применим к ним функцию
for reg in range (3):
    state = np.random.RandomState(12345)
    values = []
    count_1=0
    count_2=0
    for i in range(1000):
        target_subsample = target[reg].sample(n=500, replace=True, random_state=state)
        probs_subsample = predict[reg][target_subsample.index]
        values.append(profit(target_subsample,probs_subsample,200))
        if profit(target_subsample,probs_subsample,200)<0: count_2+=1
        count_1+=1
    values = pd.Series(values)
    lower_1 = values.quantile(0.975)
    lower_2 = values.quantile(0.025)
    mean = values.mean()
    print("Средняя выручка региона",reg,": ", round(mean))
    print("2,5%-квантиль региона",reg,": ", round(lower_2))
    print("97,5%-квантиль региона",reg,": ", round(lower_1))
    print("Доля скважин с убытком в регионе ",reg,": ",count_2/count_1 )
    print('')

Средняя выручка региона 0 :  425938527
2,5%-квантиль региона 0 :  -102090095
97,5%-квантиль региона 0 :  947976354
Доля скважин с убытком в регионе  0 :  0.06

Средняя выручка региона 1 :  515222773
2,5%-квантиль региона 1 :  68873225
97,5%-квантиль региона 1 :  931547591
Доля скважин с убытком в регионе  1 :  0.01

Средняя выручка региона 2 :  436040129
2,5%-квантиль региона 2 :  -136868692
97,5%-квантиль региона 2 :  966689525
Доля скважин с убытком в регионе  2 :  0.065



### Вывод

По итогам исследования мы можем наблюдать, что средняя выручка региона 1 является самой высокой и равна 515222773, также стоит отметить, что в данном регионе самая низкая доля скважин с убыток. Она составляет 0.01 против 0.06 и 0.065 в других регионах.
Мы видим, что в регионе 1 - 95% выручки будет выше 150785740, а 97.5% выручки выше 68873225. Это единственный регион, который удовлетворяет нашим условиям.